In [ ]:
# 설치
! pip install smolagents
# 마지막 릴리스 대신 소스에서 설치하려면 위 명령을 주석 처리하고 다음 명령의 주석을 해제하십시오.
# ! pip install git+https://github.com/huggingface/smolagents.git

# 에이전트 RAG

## 검색 증강 생성(RAG) 소개

검색 증강 생성(RAG)은 대규모 언어 모델의 강력한 기능과 외부 지식 검색을 결합하여 보다 정확하고 사실적이며 상황에 맞는 응답을 생성합니다. 핵심적으로 RAG는 "LLM을 사용하여 사용자 쿼리에 답변하되 지식 기반에서 검색된 정보를 기반으로 답변하는 것"입니다.

### RAG를 사용하는 이유

RAG는 바닐라 또는 미세 조정된 LLM을 사용하는 것보다 몇 가지 중요한 이점을 제공합니다.

1. **사실 기반**: 검색된 사실에 응답을 고정하여 환각을 줄입니다.
2. **도메인 특화**: 모델 재훈련 없이 도메인별 지식을 제공합니다.
3. **지식 최신성**: 모델의 훈련 차단 시점 이후의 정보에 액세스할 수 있습니다.
4. **투명성**: 생성된 콘텐츠에 대한 출처 인용을 가능하게 합니다.
5. **제어**: 모델이 액세스할 수 있는 정보에 대한 세분화된 제어를 제공합니다.

### 기존 RAG의 한계

이점에도 불구하고 기존 RAG 접근 방식은 몇 가지 문제에 직면합니다.

- **단일 검색 단계**: 초기 검색 결과가 좋지 않으면 최종 생성이 어려움을 겪습니다.
- **쿼리-문서 불일치**: 사용자 쿼리(종종 질문)가 답변이 포함된 문서(종종 진술)와 잘 일치하지 않을 수 있습니다.
- **제한된 추론**: 단순한 RAG 파이프라인은 다단계 추론이나 쿼리 구체화를 허용하지 않습니다.
- **컨텍스트 창 제약**: 검색된 문서는 모델의 컨텍스트 창 내에 있어야 합니다.

## 에이전트 RAG: 더 강력한 접근 방식

검색 기능을 갖춘 에이전트인 **에이전트 RAG** 시스템을 구현하여 이러한 한계를 극복할 수 있습니다. 이 접근 방식은 RAG를 엄격한 파이프라인에서 대화형 추론 기반 프로세스로 전환합니다.

### 에이전트 RAG의 주요 이점

검색 도구를 갖춘 에이전트는 다음을 수행할 수 있습니다.

1. ✅ **최적화된 쿼리 공식화**: 에이전트는 사용자 질문을 검색 친화적인 쿼리로 변환할 수 있습니다.
2. ✅ **다중 검색 수행**: 에이전트는 필요에 따라 반복적으로 정보를 검색할 수 있습니다.
3. ✅ **검색된 콘텐츠에 대한 추론**: 에이전트는 여러 출처에서 분석, 종합 및 결론을 도출할 수 있습니다.
4. ✅ **자가 비판 및 개선**: 에이전트는 검색 결과를 평가하고 접근 방식을 조정할 수 있습니다.

이 접근 방식은 고급 RAG 기술을 자연스럽게 구현합니다.
- **가상 문서 임베딩(HyDE)**: 사용자 쿼리를 직접 사용하는 대신 에이전트가 검색에 최적화된 쿼리를 공식화합니다([논문 참조](https://huggingface.co/papers/2212.10496)).
- **자가 쿼리 개선**: 에이전트는 초기 결과를 분석하고 개선된 쿼리로 후속 검색을 수행할 수 있습니다([기술 참조](https://docs.llamaindex.ai/en/stable/examples/evaluation/RetryQuery/)).

## 에이전트 RAG 시스템 구축

완전한 에이전트 RAG 시스템을 단계별로 구축해 보겠습니다. Hugging Face Transformers 라이브러리에 대한 질문에 해당 문서에서 정보를 검색하여 답변할 수 있는 에이전트를 만들 것입니다.

아래 코드 스니펫을 따라 하거나 smolagents GitHub 리포지토리의 전체 예제를 확인할 수 있습니다: [examples/rag.py](https://github.com/huggingface/smolagents/blob/main/examples/rag.py).

### 1단계: 필요한 종속성 설치

먼저 필요한 패키지를 설치해야 합니다.

```bash
pip install smolagents pandas langchain langchain-community sentence-transformers datasets python-dotenv rank_bm25 --upgrade
```

Hugging Face의 추론 API를 사용하려면 API 토큰을 설정해야 합니다.

In [ ]:
# 환경 변수 로드 (HF_TOKEN 포함)
from dotenv import load_dotenv
load_dotenv()

### 2단계: 지식 기반 준비

Hugging Face 문서가 포함된 데이터 세트를 사용하고 검색을 위해 준비합니다.

In [ ]:
import datasets
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever

# Hugging Face 문서 데이터 세트 로드
knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

# Transformers 문서만 포함하도록 필터링
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))

# 데이터 세트 항목을 메타데이터가 있는 Document 개체로 변환
source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

# 더 나은 검색을 위해 문서를 더 작은 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 청크당 문자 수
    chunk_overlap=50,  # 컨텍스트 유지를 위한 청크 간 중복
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],  # 분할 우선 순위
)
docs_processed = text_splitter.split_documents(source_docs)

print(f"지식 기반이 {len(docs_processed)}개의 문서 청크로 준비되었습니다.")

### 3단계: 검색기 도구 만들기

이제 에이전트가 지식 기반에서 정보를 검색하는 데 사용할 수 있는 사용자 지정 도구를 만들 것입니다.

In [ ]:
from smolagents import Tool

class RetrieverTool(Tool):
    name = "retriever"
    description = "의미론적 검색을 사용하여 쿼리에 답변하는 데 가장 관련성이 높은 트랜스포머 문서 부분을 검색합니다."
    inputs = {
        "query": {
            "type": "string",
            "description": "수행할 쿼리입니다. 대상 문서와 의미론적으로 가까워야 합니다. 질문보다는 긍정문을 사용하십시오.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        # 처리된 문서로 검색기 초기화
        self.retriever = BM25Retriever.from_documents(
            docs, k=10  # 가장 관련성이 높은 상위 10개 문서 반환
        )

    def forward(self, query: str) -> str:
        """제공된 쿼리를 기반으로 검색을 실행합니다."""
        assert isinstance(query, str), "검색 쿼리는 문자열이어야 합니다."

        # 관련 문서 검색
        docs = self.retriever.invoke(query)

        # 가독성을 위해 검색된 문서 형식 지정
        return "\n검색된 문서:\n" + "".join(
            [
                f"\n\n===== 문서 {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

# 처리된 문서로 검색기 도구 초기화
retriever_tool = RetrieverTool(docs_processed)

> [!TIP]
> 단순성과 속도를 위해 어휘 검색 방법인 BM25를 사용하고 있습니다. 프로덕션 시스템의 경우 더 나은 검색 품질을 위해 임베딩을 사용한 의미론적 검색을 사용하는 것이 좋습니다. 고품질 임베딩 모델은 [MTEB 리더보드](https://huggingface.co/spaces/mteb/leaderboard)를 확인하십시오.

### 4단계: 고급 검색 에이전트 만들기

이제 검색기 도구를 사용하여 질문에 답변할 수 있는 에이전트를 만들 것입니다.

In [ ]:
from smolagents import InferenceClientModel, CodeAgent

# 검색기 도구로 에이전트 초기화
agent = CodeAgent(
    tools=[retriever_tool],  # 에이전트에서 사용할 수 있는 도구 목록
    model=InferenceClientModel(),  # 기본 모델 "Qwen/Qwen2.5-Coder-32B-Instruct"
    max_steps=4,  # 추론 단계 수 제한
    verbosity_level=2,  # 자세한 에이전트 추론 표시
)

# 특정 모델을 사용하려면 다음과 같이 지정할 수 있습니다.
# model=InferenceClientModel(model_id="meta-llama/Llama-3.3-70B-Instruct")

> [!TIP]
> 추론 공급자는 서버리스 추론 파트너가 제공하는 수백 개의 모델에 대한 액세스를 제공합니다. 지원되는 공급자 목록은 [여기](https://huggingface.co/docs/inference-providers/index)에서 찾을 수 있습니다.

### 5단계: 에이전트를 실행하여 질문에 답변하기

에이전트를 사용하여 Transformers에 대한 질문에 답변해 보겠습니다.

In [ ]:
# 정보 검색이 필요한 질문하기
question = "트랜스포머 모델 학습의 경우 순방향 패스와 역방향 패스 중 어느 것이 더 느린가요?"

# 에이전트를 실행하여 답변 얻기
agent_output = agent.run(question)

# 최종 답변 표시
print("\n최종 답변:")
print(agent_output)

## 에이전트 RAG의 실제 적용 사례

에이전트 RAG 시스템은 다양한 사용 사례에 적용할 수 있습니다.

1. **기술 문서 지원**: 사용자가 복잡한 기술 문서를 탐색하는 데 도움을 줍니다.
2. **연구 논문 분석**: 과학 논문에서 정보를 추출하고 종합합니다.
3. **법률 문서 검토**: 법률 문서에서 관련 판례 및 조항을 찾습니다.
4. **고객 지원**: 제품 설명서 및 지식 기반을 기반으로 질문에 답변합니다.
5. **교육 튜터링**: 교과서 및 학습 자료를 기반으로 설명을 제공합니다.

## 결론

에이전트 RAG는 기존 RAG 파이프라인보다 훨씬 발전된 형태입니다. LLM 에이전트의 추론 기능과 검색 시스템의 사실 기반을 결합하여 보다 강력하고 유연하며 정확한 정보 시스템을 구축할 수 있습니다.

여기서 보여드린 접근 방식은 다음과 같습니다.
- 단일 단계 검색의 한계를 극복합니다.
- 지식 기반과의 보다 자연스러운 상호 작용을 가능하게 합니다.
- 자가 비판 및 쿼리 개선을 통해 지속적인 개선을 위한 프레임워크를 제공합니다.

자신만의 에이전트 RAG 시스템을 구축할 때 다양한 검색 방법, 에이전트 아키텍처 및 지식 출처를 실험하여 특정 사용 사례에 가장 적합한 구성을 찾아보십시오.